In [1]:
import geopandas as gpd
import ipyleaflet
import urllib
import urllib.request
from zipfile import ZipFile
import io
import requests
import boto3

"""Accessing the S3 buckets using boto3 client"""
s3_url = "s3://dejas-datasets/REPTILES.zip"
s3_client = boto3.client('s3')
s3_bucket_name='REPTILES.zip'
s3 = boto3.resource('s3',
                    aws_access_key_id= 'YOUR_ACCESS_KEY_ID',
                    aws_secret_access_key='YOUR_SECRET_ACCESS_KEY')

In [ ]:
def endangered_species_mapping(species_shapefile, aus_shapefile):
    s3_resource = boto3.resource('s3')
    zip_obj = s3_resource.Object(bucket_name="dejas-datasets", key=f"{species_shapefile}.zip")
    try:
        buffer = BytesIO(zip_obj.get()["Body"].read())
        z = ZipFile(buffer)
        filenames = [y for y in sorted(z.namelist()) for ending in ['dbf', 'prj', 'shp', 'shx'] if y.endswith(ending)]
        gpds = []
        for file in filenames:
            if file.endswith(".shp"):
                z.extractall(path='./dataset/')
                species_gpd = gpd.read_file(f"./dataset/{file}")
                aus_url = aus_shapefile
                local_path = './dataset/'
                r = requests.get(aus_url)
                aus_z = ZipFile(io.BytesIO(r.content))
                aus_z.extractall(path=local_path) # extract to folder
                filenames = [y for y in sorted(aus_z.namelist()) for ending in ['dbf', 'prj', 'shp', 'shx'] if y.endswith(ending)] 
                dbf, prj, shp, shx = [filename for filename in filenames]
                aus = gpd.read_file(local_path + shp)
                aus_new = aus[~aus.isna()]
                aus_new = aus_new.to_crs("EPSG:4326")
                species_gpd_new = species_gpd[species_gpd.contains(aus_new)]
                gpds.append(species_gpd_new)
        all_gpd = gpd.GeoDataFrame(pd.concat(gpds))
        species_gpd_new.to_file(f"./dataset/{species_shapefile}")
        return species_gpd_new
    except ClientError as ex:
        if ex.response['Error']['Code'] == 'NoSuchKey':
            print('No object found - returning empty')
            return dict()
        else:
            return None

endangered_species = ['PLANTS']
for species in endangered_species:
    species_gdf = endangered_species_mapping(species, "https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files/SA2_2021_AUST_SHP_GDA2020.zip")

In [2]:
Working_Dir = "./"
DATA_DIR = f"{Working_Dir}dataset/"
SPECIES_DIR = f"{DATA_DIR}/endangered_species/"

In [3]:
reptile_part1 = gpd.read_file(f"{Working_Dir}REPTILES/REPTILES_PART1.shp")
# reptile_part2 = gpd.read_file(f"{Working_Dir}REPTILES/REPTILES_PART2.shp")

endangered_species = ['AMPHIBIANS', 'MAMMALS', 'PLANTS', 'REPTILES']

# amp = ZipFile(f"{SPECIES_DIR}/{endangered_species[0]}.zip")
# mammals = ZipFile(f"{SPECIES_DIR}/{endangered_species[1]}.zip")
# plants = ZipFile(f"{SPECIES_DIR}/{endangered_species[2]}.zip")
# reptiles_zip = ZipFile(open(f"{SPECIES_DIR}/{endangered_species[3]}.zip", 'rb'))
# filenames = [y for y in sorted(reptiles_zip.namelist()) for ending in ['dbf', 'prj', 'shp', 'shx'] if y.endswith(ending)]
# print(filenames)
# dbfname, _, shpname, _, shxname = reptiles_zip.namelist()
# print(reptiles.open("REPTILES_PART1.shp", "r"))
# print(reptiles_zip.namelist())
# reptiles_part1 = gpd.read_file(reptiles_zip.read("REPTILES_PART1.shp"))
# reptiles_part2 = gpd.read_file(reptiles.open("REPTILES_PART2.shp"))

In [4]:
reptile_part1

,OBJECTID,id_no,sci_name,presence,origin,seasonal,compiler,yrcompiled,citation,subspecies,...,order_,family,genus,category,marine,terrestial,freshwater,SHAPE_Leng,SHAPE_Area,geometry
0,1,83777229,Varanus acanthurus,1,1,1,"Stewart MacDonald, Reid Tingley",2017,"IUCN, Stewart MacDonald, Reid Tingley",None,...,SQUAMATA,VARANIDAE,Varanus,LC,false,true,false,163.801096,238.662979,"MULTIPOLYGON (((123.49866 -17.40178, 123.49809..."
1,2,203303,Mastigodryas pulchriceps,1,1,1,"Sarah Hanson, Hedges Lab, Temple University",2015,"Center for Biodiversity, Temple University",None,...,SQUAMATA,COLUBRIDAE,Mastigodryas,LC,false,true,false,29.541178,9.623437,"MULTIPOLYGON (((-78.19936 2.93807, -78.20415 2..."
2,3,192191,Cylindrophis engkariensis,1,1,1,None,2011,IUCN,None,...,SQUAMATA,CYLINDROPHIIDAE,Cylindrophis,DD,false,true,false,0.556758,0.024667,"POLYGON ((111.78827 1.37345, 111.78907 1.37345..."
3,4,13288920,Chamaelycus fasciatus,1,1,1,IUCN,2019,IUCN,None,...,SQUAMATA,LAMPROPHIIDAE,Chamaelycus,LC,false,true,false,121.792063,236.327312,"MULTIPOLYGON (((9.54644 0.27002, 9.54139 0.267..."
4,5,48280071,Bachia peruana,1,1,1,"Sarah Hanson, Hedges Lab, Temple University",2015,"Center for Biodiversity, Temple University",None,...,SQUAMATA,GYMNOPHTHALMIDAE,Bachia,LC,false,true,false,17.512170,12.438952,"POLYGON ((-72.36164 -9.83140, -72.67068 -10.45..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6844,6845,56059651,Liolaemus escarchadosi,1,1,1,"Sarah Hanson, Hedges Lab, Temple University",2015,NatureServe; IUCN (International Union for Con...,None,...,SQUAMATA,LIOLAEMIDAE,Liolaemus,LC,false,true,false,8.456513,3.250214,"POLYGON ((-71.85920 -51.14484, -71.92441 -51.1..."
6845,6846,56059651,Liolaemus escarchadosi,1,1,1,"Sarah Hanson, Hedges Lab, Temple University",2015,NatureServe; IUCN (International Union for Con...,None,...,SQUAMATA,LIOLAEMIDAE,Liolaemus,LC,false,true,false,8.456513,3.250214,"POLYGON ((-71.85920 -51.14484, -71.92441 -51.1..."
6846,6847,102966856,Carlia rufilatus,1,1,1,"Stewart MacDonald, Reid Tingley",2017,"IUCN, Stewart MacDonald, Reid Tingley",None,...,SQUAMATA,SCINCIDAE,Carlia,LC,false,true,false,92.740910,25.132750,"MULTIPOLYGON (((124.27996 -16.40211, 124.28020..."
6847,6848,74875182,Cadea blanoides,1,1,1,"Sarah Hanson, Hedges Lab, Temple University",2016,"Center for Biodiversity, Temple University",None,...,SQUAMATA,CADEIDAE,Cadea,LC,false,true,false,16.441241,2.063183,"MULTIPOLYGON (((-80.70601 23.08468, -80.70625 ..."


In [5]:
reptile_part1.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 6849 entries, 0 to 6848
Data columns (total 30 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   OBJECTID    6849 non-null   int64   
 1   id_no       6849 non-null   int64   
 2   sci_name    6849 non-null   object  
 3   presence    6849 non-null   int64   
 4   origin      6849 non-null   int64   
 5   seasonal    6849 non-null   int64   
 6   compiler    6746 non-null   object  
 7   yrcompiled  6849 non-null   int64   
 8   citation    6849 non-null   object  
 9   subspecies  13 non-null     object  
 10  subpop      15 non-null     object  
 11  source      1342 non-null   object  
 12  island      880 non-null    object  
 13  tax_comm    3 non-null      object  
 14  dist_comm   103 non-null    object  
 15  generalisd  6849 non-null   int64   
 16  legend      6849 non-null   object  
 17  kingdom     6849 non-null   object  
 18  phylum      6849 non-null   object  
 19

In [6]:
aus_url = "https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files/SA2_2021_AUST_SHP_GDA2020.zip"
local_path = './dataset/'
print('Downloading shapefile...')
r = requests.get(aus_url)
z = ZipFile(io.BytesIO(r.content))
print("Done")
# z.extractall(path=local_path) # extract to folder
filenames = [y for y in sorted(z.namelist()) for ending in ['dbf', 'prj', 'shp', 'shx'] if y.endswith(ending)] 
print(filenames)

Done
['SA2_2021_AUST_GDA2020.dbf', 'SA2_2021_AUST_GDA2020.prj', 'SA2_2021_AUST_GDA2020.shp', 'SA2_2021_AUST_GDA2020.shx']


In [7]:
dbf, prj, shp, shx = [filename for filename in filenames]
aus = gpd.read_file(local_path + shp)
print("Shape of the dataframe: {}".format(aus.shape))
print("Projection of dataframe: {}".format(aus.crs))
aus.tail() #last 5 records in dataframe

Shape of the dataframe: (2473, 17)
Projection of dataframe: EPSG:7844


,SA2_CODE21,SA2_NAME21,CHG_FLAG21,CHG_LBL21,SA3_CODE21,SA3_NAME21,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geometry
2468,901031003,Jervis Bay,0,No change,90103,Jervis Bay,901,Other Territories,9OTER,Other Territories,9,Other Territories,AUS,Australia,67.2296,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"MULTIPOLYGON (((150.69567 -35.18295, 150.69556..."
2469,901041004,Norfolk Island,0,No change,90104,Norfolk Island,901,Other Territories,9OTER,Other Territories,9,Other Territories,AUS,Australia,38.6510,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"MULTIPOLYGON (((167.96325 -29.07212, 167.96326..."
2470,997979799,Migratory - Offshore - Shipping (OT),0,No change,99797,Migratory - Offshore - Shipping (OT),997,Migratory - Offshore - Shipping (OT),99799,Migratory - Offshore - Shipping (OT),9,Other Territories,AUS,Australia,NaN,http://linked.data.gov.au/dataset/asgsed3/SA2/...,None
2471,999999499,No usual address (OT),0,No change,99999,No usual address (OT),999,No usual address (OT),99499,No usual address (OT),9,Other Territories,AUS,Australia,NaN,http://linked.data.gov.au/dataset/asgsed3/SA2/...,None
2472,ZZZZZZZZZ,Outside Australia,1,New,ZZZZZ,Outside Australia,ZZZ,Outside Australia,ZZZZZ,Outside Australia,Z,Outside Australia,ZZZ,Outside Australia,NaN,http://linked.data.gov.au/dataset/asgsed3/SA2/...,None


In [8]:
aus_new = aus[~aus.isna()]
print("Shape of the dataframe: {}".format(aus_new.shape))
print("Projection of dataframe: {}".format(aus_new.crs))

Shape of the dataframe: (2473, 17)
Projection of dataframe: EPSG:7844


In [9]:
aus_new_4326 = aus_new.to_crs("EPSG:4326")
print("Projection of dataframe: {}".format(reptile_part1.crs))
print("Projection of dataframe: {}".format(aus_new_4326.crs))

Projection of dataframe: EPSG:4326
Projection of dataframe: EPSG:4326


In [10]:
reptile_new = reptile_part1.aligns(aus_new_4326, how='intersection')
reptile_new.shape

(76705, 46)

In [11]:
reptile_new.to_file(f"{DATA_DIR}aus_reptile")

In [ ]:
from ipyleaflet  import Map, GeoData, basemaps, LayersControl

In [ ]:
m = Map(
    basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    center=(48.204793, 350.121558),
    zoom=3
    )
m

In [ ]:
new_rep = GeoData(geo_dataframe = reptile_new)
m.addLayer(new_rep)
m

In [ ]:
reptile_new.head()

In [ ]:
new_rep